In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, Time, DateTime
import sqlalchemy as db

import pandas as pd
from datetime import datetime

# GTFS Files

## Trips: `trips.txt`

In [2]:
trips = pd.read_csv('trips.txt')
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 9 columns):
route_id                 847 non-null object
service_id               847 non-null object
trip_id                  847 non-null int64
trip_headsign            847 non-null object
direction_id             847 non-null int64
block_id                 847 non-null object
shape_id                 847 non-null int64
wheelchair_accessible    847 non-null int64
bikes_allowed            847 non-null int64
dtypes: int64(5), object(4)
memory usage: 59.6+ KB


In [3]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,NIS,SU,712,Gungahlin Pl,0,3,1007,1,1
1,ACTO001,SU,780,Alinga St,1,3,1003,1,1
2,ACTO001,SU,718,Gungahlin Pl,0,3,1004,1,1
3,ACTO001,SU,784,Alinga St,1,3,1003,1,1
4,ACTO001,SU,722,Gungahlin Pl,0,3,1004,1,1


### List of Routes

In [4]:
trips['route_id'].unique()

array(['NIS', 'ACTO001', 'X1', 'X2'], dtype=object)

In [5]:
trips = trips[trips['route_id'] == 'ACTO001']

### Keep Columns

In [6]:
trips = trips[['route_id' ,'service_id', 'trip_id', 'trip_headsign', 'direction_id']]

## Stop Times: `stop_times.txt`

In [7]:
stop_times = pd.read_csv('stop_times.txt')
stop_times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9724 entries, 0 to 9723
Data columns (total 9 columns):
trip_id           9724 non-null int64
arrival_time      9724 non-null object
departure_time    9724 non-null object
stop_id           9724 non-null int64
stop_sequence     9724 non-null int64
stop_headsign     9724 non-null object
pickup_type       9724 non-null int64
drop_off_type     9724 non-null int64
timepoint         9724 non-null int64
dtypes: int64(6), object(3)
memory usage: 683.8+ KB


In [8]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint
0,712,07:36:16,07:36:36,8111,1,Well Station Drive,0,0,1
1,712,07:38:20,07:38:40,8109,2,Nullarbor Avenue,0,0,1
2,712,07:39:51,07:40:11,8107,3,Mapleton Avenue,0,0,1
3,712,07:41:45,07:42:05,8105,4,Manning Clark Crescent,0,0,1
4,712,07:43:43,07:43:43,8100,5,Gungahlin Place,0,0,1


### Clean Arrival Time and Departure Time

In [9]:
stop_times['arrival_time'] = stop_times['arrival_time'].apply(lambda x: x.split(':'))
stop_times['departure_time'] = stop_times['departure_time'].apply(lambda x: x.split(':'))

In [10]:
stop_times.iloc[5095]

trip_id                       540
arrival_time         [25, 01, 04]
departure_time       [25, 01, 24]
stop_id                      8106
stop_sequence                   3
stop_headsign     Mapleton Avenue
pickup_type                     0
drop_off_type                   0
timepoint                       1
Name: 5095, dtype: object

In [11]:
for idx, val in stop_times.iterrows():
    if val['arrival_time'][0] == '24':
        stop_times.loc[idx, 'arrival_time'][0] = '00'
#         print(val['arrival_time'])
    elif val['arrival_time'][0] == '25':
        stop_times.loc[idx, 'arrival_time'][0] = '00'
#         print(val['arrival_time'])
        
for idx, val in stop_times.iterrows():
    if val['departure_time'][0] == '24':
        stop_times.loc[idx, 'departure_time'][0] = '00'
#         print(val['departure_time'])
    elif val['departure_time'][0] == '25':
        stop_times.loc[idx, 'departure_time'][0] = '01'
#         print(val['departure_time'])

In [12]:
stop_times['arrival_time'] = stop_times['arrival_time'].apply(lambda x: ':'.join(x))
stop_times['departure_time'] = stop_times['departure_time'].apply(lambda x: ':'.join(x))

In [13]:
stop_times['arrival_time'] = stop_times['arrival_time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())
stop_times['departure_time'] = stop_times['departure_time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

### Keep Columns

In [14]:
stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_headsign']]

# PostgreSQL

## Tables

In [15]:
# check trips dataset
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id
1,ACTO001,SU,780,Alinga St,1
2,ACTO001,SU,718,Gungahlin Pl,0
3,ACTO001,SU,784,Alinga St,1
4,ACTO001,SU,722,Gungahlin Pl,0
5,ACTO001,SU,788,Alinga St,1


In [16]:
# check stop_times dataset
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign
0,712,07:36:16,07:36:36,8111,1,Well Station Drive
1,712,07:38:20,07:38:40,8109,2,Nullarbor Avenue
2,712,07:39:51,07:40:11,8107,3,Mapleton Avenue
3,712,07:41:45,07:42:05,8105,4,Manning Clark Crescent
4,712,07:43:43,07:43:43,8100,5,Gungahlin Place


In [17]:
# SQL database information
db_uri = 'postgresql://postgres@localhost:5432/noelangelo'
engine = create_engine(db_uri)

meta = MetaData(engine)

### Trips Table

In [36]:
t1 = Table('sched_trips', meta,
           Column('Route ID', String),
           Column('Service ID',String),
           Column('Trip ID',String, primary_key=True),
           Column('Trip Headsign',String),
           Column('Direction ID',Integer))

# t1.create()

### Stop Times Table

In [37]:
t2 = Table('sched_stop_times', meta,
           Column('Trip ID', String),
           Column('Arrival Time',Time),
           Column('Departure Time',Time),
           Column('Stop ID',String),
           Column('Stop Sequence',Integer),
           Column('Stop Headsign',String))

### Real-time Table

In [18]:
t3 = Table('gtfsr_feed', meta,
                 Column('Arrival Time', Integer),
                 Column('Arrival Delay', DateTime),
                 Column('Arrival Uncertainty', Integer),
                 Column('Departure Delay', Integer),
                 Column('Departure Time', DateTime),
                 Column('Departure Uncertainty', Integer),
                 Column('Schedule Relationship', String),
                 Column('Stop ID', String),
                 Column('Stop Sequence', Integer),
                 Column('Trip ID', String),
                 Column('Request Timestamp', DateTime),
                 Column('Delay', Integer),
                 Column('Feed ID', Integer))

### Create Tables

In [38]:
t1.create() # create sched_trips table
t2.create() # create sched_stop_times table

In [19]:
t3.create() # create realtime feed table

In [46]:
trips.columns = ['Route ID', 'Service ID', 'Trip ID', 'Trip Headsign', 'Direction ID']

In [45]:
stop_times.columns = ['Trip ID', 'Arrival Time', 'Departure Time', 'Stop ID', 'Stop Sequence', 'Stop Headsign']

### Insert Records

In [48]:
conn = engine.connect()

In [50]:
conn.execute(t1.insert(trips.to_dict(orient='records')))

In [51]:
conn.execute(t2.insert(stop_times.to_dict(orient='records')))